In [1]:
# all internal libraries
import time
import datetime

# dependencies
import networkx as nx
import simpy
import shapely
import pandas as pd

# our software
import dtv_backend.fis
import opentnsim.core
import openclsim.core

%matplotlib inline

In [2]:
# link to the latets version of the network
url = 'https://zenodo.org/record/3981105/files/network_digital_twin_v0.1.yaml'
G = dtv_backend.fis.load_fis_network(url)

## Simple test

Load the network and create an opentnsim simulation. 

In [3]:
# get an edge from the network
edge = list(G.edges())[0]

In [4]:
# we're sailing from A to B
a = G.nodes[edge[0]]
b = G.nodes[edge[1]]

In [5]:
# Define the simulation environment
env = simpy.Environment()

In [6]:
# use an epoch based time
# See: https://en.wikipedia.org/wiki/Epoch_(computing)
simulation_start = datetime.datetime(2018, 7, 1)
env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
# store the epoch so we can convert back
env.epoch = time.mktime(simulation_start.timetuple())

# Add graph to environment
env.FG = G

In [7]:
# Make a general vessel class out of mix-ins
TransportResource = type(
    'TransportResource', 
    (
        openclsim.core.Identifiable, 
        opentnsim.core.ContainerDependentMovable, 
        opentnsim.core.HasResource, 
        opentnsim.core.Routeable
    ), 
    {}
)



# For testing purposes we only want v to be 1
def compute_v_provider(v_empty, v_full):
    return lambda x: 1

data_vessel = {"env": None,
               "name": "Vessel number 1",
               "route": None,
               "geometry": a['geometry'],  # lon, lat
               "capacity": 1_000,
               "compute_v": compute_v_provider(v_empty=1, v_full=1)}

# create the transport processing resource
vessel = TransportResource(**data_vessel)


In [8]:
# Now define the sites (A and B)
Node = type(
    'Site', 
    (
        openclsim.core.Identifiable, 
        opentnsim.core.Log, 
        opentnsim.core.Locatable, 
        opentnsim.core.HasResource
    ), 
    {}
)

data_node_1 = {"env": [],
               "name": "Node 1",
               "geometry": shapely.geometry.Point(a['X'], a['Y'])}
data_node_2 = {"env": [],
               "name": "Node 2",
               "geometry": shapely.geometry.Point(b['X'], b['Y'])}

node_1 = Node(**data_node_1)
node_2 = Node(**data_node_2)

nodes = [node_1, node_2]

In [9]:
# define the route along which we're going to sail
path = [edge]

In [10]:
# # Add environment and path to the vessel
vessel.env = env
vessel.route = path[0]
# these node should be different.
vessel.route

(8864523.0, 8864358.0)

In [11]:
# Now we can run a first simulation
def start(env, vessel):
    yield from vessel.move()
    vessel.log_entry("Stop sailing", env.now, "", vessel.geometry)
        
env.process(start(env, vessel))
env.run()


In [12]:
# The vessel should sail from A to B
pd.DataFrame(vessel.log)

,Message,Timestamp,Value,Geometry
0,Sailing from node 8864523.0 to node 8864358.0 ...,2018-07-01 00:00:00.000000,0,POINT (4.69599104281199 51.7190069976013)
1,Sailing from node 8864523.0 to node 8864358.0 ...,2018-07-01 00:03:06.483382,0,POINT (4.69594354276411 51.7206871440897)
2,Stop sailing,2018-07-01 00:03:06.483382,,POINT (4.69594354276411 51.7206871440897)
